In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.simplefilter('ignore')

In [2]:
from sklearn.linear_model import Ridge ,LinearRegression

In [3]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [5]:
def Train_5Fold_Stacking(X,y,Test,kfold) :
  final_preds = [] ; err_cb = []
  oof_stack = np.zeros(len(X)) ;
  for fold,(train_index, test_index) in enumerate(kfold.split(X,y)):
      X_train, X_test = X.values[train_index], X.values[test_index]
      y_train, y_test = y.values[train_index], y.values[test_index]

      model = Ridge(alpha=0.01,random_state=42)
      model.fit(X_train,y_train)
      preds=model.predict(X_test)
      preds = np.clip(preds,a_min=0,a_max=500)
      oof_stack[test_index] = preds
      err_cb.append(mean_squared_error(y_test,preds,squared=False))
      
      test_pred = model.predict(Test.values)
      final_preds.append(test_pred)
  
  print(2*'--------------------------------------')
  print(f"Stacking RMSE :" ,np.mean(err_cb))
  return oof_stack,np.mean(final_preds,axis=0)

In [6]:
stacking_train_H1 = pd.read_csv('Hela_App1_stacking_train.csv')
stacking_test_H1 = pd.read_csv('Hela_App1_stacking_test.csv')

stacking_train_H2 = pd.read_csv('Hela_App2_stacking_train.csv')
stacking_test_H2 = pd.read_csv('Hela_App2_stacking_test.csv')

stacking_train_A = pd.read_csv('azer_stacking_train.csv')
stacking_test_A = pd.read_csv('azer_stacking_test.csv')

In [7]:
stacking_train_H1 = stacking_train_H1.sort_values('ID').reset_index(drop=True)
stacking_train_H2 = stacking_train_H2.sort_values('ID').reset_index(drop=True)
stacking_train_A = stacking_train_A.sort_values('ID').reset_index(drop=True)

In [8]:
stacking_test_H1 = stacking_test_H1.sort_values('ID').reset_index(drop=True)
stacking_test_H2 = stacking_test_H2.sort_values('ID').reset_index(drop=True)
stacking_test_A = stacking_test_A.sort_values('ID').reset_index(drop=True)

In [9]:
stacking_train = pd.concat([stacking_train_A,stacking_train_H1.drop(['ID','ref_pm2_5'],1),
                            stacking_train_H2.drop(['ID','ref_pm2_5'],1),
                            ],axis=1)

In [10]:
stacking_test= pd.concat([stacking_test_A,stacking_test_H2.drop(['ID'],1),
                            stacking_test_H1.drop(['ID',],1),
                            ],axis=1)

In [11]:
cols = ['azer_preds_cat_2', 'azer_preds_xgb', 
        'Hela_App2_preds_cat_2', 'Hela_App2_preds_xgb', 
        'Hela_App1_preds_cat',  'Hela_App1_preds_xgb',]

X , y , Test = stacking_train[cols] , stacking_train['ref_pm2_5'] , stacking_test[cols]
KFOLD = KFold(n_splits=5,random_state=1901,shuffle=True)

oof_stack,stack_preds = Train_5Fold_Stacking(X=X,y=y,Test=Test,kfold=KFOLD)

----------------------------------------------------------------------------
Stacking RMSE : 10.91059241063979


In [12]:
print('Stacking OOF',mean_squared_error(y,oof_stack,squared=False))

Stacking OOF 11.120406649210725


In [13]:
SUB_FILE_NAME = 'Final_Submission.csv' ;sub_df = stacking_test[['ID']].copy() ; sub_df['ref_pm2_5'] = stack_preds
sub_df.to_csv(SUB_FILE_NAME, index=False)
sub_df.head(10)

,ID,ref_pm2_5
0,ID_00OZLF7X,40.025604
1,ID_00ZI0D98,40.227027
2,ID_017GTLAU,42.420329
3,ID_01IBM7T2,39.165266
4,ID_01II27D4,66.152040
5,ID_01PKRNEP,23.211764
6,ID_021PCRSH,14.506825
7,ID_0274H7B6,73.152953
8,ID_02NFLBYH,32.162417
9,ID_032LIJKT,25.598222
